In [41]:
import requests
from bs4 import BeautifulSoup
import nltk

In [42]:



url = 'https://www.yarsabazar.com/'  
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}



In [43]:
def ParseHTML(url,headers):
    response = requests.get(url,headers= headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        return soup
        
    else:
        print('Failed to retrieve the page')
        return -1
        

In [44]:
soup = ParseHTML(url,headers)

In [51]:
def SplitDocumentsByTags(soup):
    chunks = []

    for heading in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6','div']):
        heading_text = heading.get_text()
        sibling = heading.find_next_sibling()
        content = []

        while sibling and sibling.name not in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6''div']:
            content.append(sibling.get_text())
            sibling = sibling.find_next_sibling()

        chunks.append((heading_text, " ".join(content)))
    return chunks

In [52]:
SplitDocumentsByTags(soup)

[('',
  'HelpLoginSign Up Search your desired ProductsVendorsCategoriesSearch your desired ProductsVendorsCategoriesIndustriesSee allAgriculture, Livestock and HerbsFurniture, Wood and MetalPaper, Plastics and PackagingClothing, Jewelry and TextileFood, Drinks and GroceryMedical, Pharma and ChemicalsElectronics, IT and NetworkingPower, Energy and ElectricityTools, Machinery and AutomobileBuilding, Construction and PaintsEntertainment, Media and ArtsTourism, Hospitality and RecreationSports Goods, Toys and GamesProduct Rental and LeasingTransportation and LogisticsR&D and Testing LabsBooks and StationeryCosmetics and Personal CareLooking for something?Please provide your requirements, and we will contact you to assist in fulfilling them.Ask for QuotationsSell on Yarsa BazarCreate a seller account and start expanding your business on Yarsa BazarUpload my ProductsAgriculture, Livestock and HerbsBrowse ProductsSiddhartha Bakhara Ko Dana| Goat Feed(Contact the seller for pricing info)Min Or

In [49]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
def SplitDocumentsBySlidingWindow(soup):
    text = " ".join([p.get_text() for p in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6','p','div'])])

    # Split text into sentences
    sentences = sent_tokenize(text)

    # Define window size and stride
    window_size = 3
    stride = 2

    chunks = []

    # Create sliding windows of sentences
    for i in range(0, len(sentences), stride):
        window = sentences[i:i + window_size]
        if window:
            chunks.append(" ".join(window))
    return chunks

[nltk_data] Downloading package punkt to /home/prayash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
SplitDocumentsBySlidingWindow(soup)

[' HelpLoginSign Up HelpLoginSign Up HelpLoginSign Up   Search your desired ProductsVendorsCategoriesSearch your desired ProductsVendorsCategories Search your desired ProductsVendorsCategories Search your desired ProductsVendorsCategories Search your desired ProductsVendorsCategories Search your desired ProductsVendorsCategories Search your desired ProductsVendorsCategories ProductsVendorsCategories Search your desired ProductsVendorsCategories ProductsVendorsCategories IndustriesSee all Industries Agriculture, Livestock and HerbsFurniture, Wood and MetalPaper, Plastics and PackagingClothing, Jewelry and TextileFood, Drinks and GroceryMedical, Pharma and ChemicalsElectronics, IT and NetworkingPower, Energy and ElectricityTools, Machinery and AutomobileBuilding, Construction and PaintsEntertainment, Media and ArtsTourism, Hospitality and RecreationSports Goods, Toys and GamesProduct Rental and LeasingTransportation and LogisticsR&D and Testing LabsBooks and StationeryCosmetics and Perso